In [19]:
import matplotlib.pyplot as plt
plt.rcParams["savefig.bbox"] = 'tight'

import torch
from torch import nn
import torchvision
import torchvision.transforms as T
import torchvision.transforms.functional as TF

import lightning.pytorch as pl
import os

In [20]:
# sets seeds for numpy, torch and python.random.
pl.seed_everything(42, workers=True)

Global seed set to 42


42

In [21]:
torch.set_float32_matmul_precision('medium')

# Transforms

In [22]:
train_transform = T.Compose([
    T.Resize((132, 132)),
    T.RandomCrop((128, 128)),
    T.RandomAdjustSharpness(sharpness_factor=4, p=0.7),
    T.RandomAutocontrast(p=0.7),
    T.ColorJitter(brightness=0.6, hue=0.3, contrast=0.6, saturation=0.6),
    T.GaussianBlur(kernel_size=3),
])

In [23]:
test_transform = T.Compose([
    T.Resize((128, 128)),
])

# Data Module

In [24]:
from data_module import TripletDataset, UdacitySimDataModule

NUM_WORKERS = os.cpu_count() # <- use all available CPU cores

udacity_sim_dm = UdacitySimDataModule(csv_path=".\data\driving_log.csv",
                                      images_dir="data\IMG",
                                      batch_size=32,
                                      num_workers=NUM_WORKERS,
                                      train_transform=train_transform,
                                      test_transform=test_transform)

# PyTorch Model

In [25]:
from model import BabyHamiltonModel
model = BabyHamiltonModel()

# Lightning Model

In [26]:
from train import LightingBabyHamiltonModel
lightning_model = LightingBabyHamiltonModel(model)

# Logger

In [27]:
from train import get_logger
logger = get_logger()

# from lightning.pytorch.loggers import TensorBoardLogger
# logger = TensorBoardLogger(save_dir=os.getcwd(), version=1, name="lightning_logs")

# Trainer

In [28]:
trainer = pl.Trainer(
    # Permanent
    default_root_dir="lightning/",
    max_epochs=10,
    devices='auto', # 1, 2, ...
    accelerator="auto", # gpu, cpu, ...
    logger=logger,
    check_val_every_n_epoch=1,
    
    # Temporary
    
    fast_dev_run=True,
    profiler="simple",
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Running in `fast_dev_run` mode: will run the requested loop using 1 batch(es). Logging and checkpointing is suppressed.


# Tuner

In [29]:
from lightning.pytorch.tuner import Tuner
tuner = Tuner(trainer)

## Learning Rate

In [32]:
lr_finder = tuner.lr_find(lightning_model, datamodule=udacity_sim_dm)

The following callbacks returned in `LightningModule.configure_callbacks` will override existing callbacks passed to Trainer: EarlyStopping, ModelCheckpoint, TQDMProgressBar
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
FIT Profiler Report

-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|  Action                                                                                                                                             	|  Mean duration (s)	|  Num calls      	|  Total time (s) 	|  Percentage %   	|
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|  Total                    

In [31]:
lr_finder.plot(suggest=True)

AttributeError: 'NoneType' object has no attribute 'plot'

In [ ]:
new_lr = lr_finder.suggestion()
lightning_model.learning_rate = new_lr
new_lr

## Batch Size

In [ ]:
bs_finder = tuner.scale_batch_size(
    lightning_model, 
    datamodule=udacity_sim_dm
)

In [ ]:
bs_finder.plot(suggest=True)

In [ ]:
new_bs = bs_finder.suggestion()
udacity_sim_dm.batch_size = new_bs
lightning_model.batch_size = new_bs
new_bs

# Training

## Fit

In [ ]:
trainer.fit(
    lightning_model, 
    datamodule=udacity_sim_dm
)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
C:\Users\prana\miniconda3\envs\baby_hamilton\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:478: PossibleUserWarning: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
  rank_zero_warn(


## Plot Metrics

In [ ]:
from utils import plot_csv_logger
plot_csv_logger(csv_path=f'{trainer.logger.log_dir}/metrics.csv')

# Testing

In [ ]:
trainer.test(
    lightning_model, 
    datamodule=udacity_sim_dm, 
    ckpt_path='best'
)

# Production

In [ ]:
path = trainer.checkpoint_callback.best_model_path
lightning_model = LightningBabyHamiltonModel.load_from_checkpoint(path)

In [ ]:
x = torch.randn(1, 64)

with torch.inference_mode():
    x = x.unsqueeze(0)
    y_hat = lightning_model(x)